### Predefined variables for assignment

In [36]:
# Define the substitution table
substitution_table = {
    "0": "a",
    "1": "0",
    "2": "9",
    "3": "e",
    "4": "6",
    "5": "3",
    "6": "f",
    "7": "5",
    "8": "1",
    "9": "d",
    "a": "c",
    "b": "7",
    "c": "b",
    "d": "4",
    "e": "2",
    "f": "8"
}
reverse_substitution_table = {v: k for k, v in substitution_table.items()}
# Define the constant matrix
constant_matrix = [
    [1, 4],
    [4, 1]
]
inverse_constant_matrix = [
    [9, 2],
    [2, 9]
]
# Define round constants
rcon1 = "1110"
rcon2 = "1010"
rcons = [rcon1, rcon2]

# SUB NIBBLE FUNCTIONS FOR BINARY 

def sub_nibbles(nibble):
    # Defining the S-Box based on the table provided
    # plaintext:mapping 
    s_box = {
        '0000': '1010', '0001': '0000', '0010': '1001', '0011': '1110',
        '0100': '0110', '0101': '0011', '0110': '1111', '0111': '0101',
        '1000': '0001', '1001': '1101', '1010': '1100', '1011': '0111',
        '1100': '1011', '1101': '0100', '1110': '0010', '1111': '1000'
    }
    return s_box[nibble]

# test run for subnibbles

print("subnibbles example: ", sub_nibbles('0011'))

def gf_multiply(a, b):
    result = 0
    for _ in range(4):  # Iterate over each bit in a and b
        if b & 1:  # Check if the least significant bit of b is set
            result ^= a  # If yes, XOR a with the result
        carry = a & 0x08  # Check if the most significant bit of a is set
        a <<= 1  # Left shift a by 1 bit
        if carry:  # If carry is set, perform modulo reduction
            a ^= 0x13  # 0x13 is the polynomial x^4 + x + 1
        b >>= 1  # Right shift b by 1 bit
    return result

subnibbles example:  1110


### Basic Conversions

In [37]:
def hex_to_bits(hex_number):
    binary_str = bin(int(hex_number, 16))[2:]
    return binary_str.zfill(16)
def bits_to_hex(binary_str):
    if len(binary_str) != 16:
        raise ValueError("Binary string must be 16 bits long")
    hex_str = hex(int(binary_str, 2))[2:]
    return hex_str.zfill(4)
def hex_to_ascii(hex_number):
    return hex_number.decode("hex")
def ascii_to_hex(ascii_str):
    return ascii_str.encode("hex")
def bin_to_ascii(bin_number):
    return bin_number.decode("binary")
def ascii_to_bin(ascii_str):
    return ascii_str.encode("binary")

### Subnibbles function

In [38]:
# nothing but reversing the nibbles dict or simply using the reverse substitution table
def inverse_subnibbles(input_bits, table):
    if len(input_bits) != 4:
        raise ValueError("Input length must be 4 hex characters")
    result = ""
    # Iterate over the input, processing 4 bits (1 nibble) at a time
    for i in input_bits:
        substituted_nibble = table[i]
        result += substituted_nibble
    return result

## Bitwise XOR

In [39]:
def AddRoundKeys(input_bits, key_bits):
    input_bits = hex_to_bits(input_bits)
    key_bits = hex_to_bits(key_bits)
    result = ""
    # Perform XOR operation bit by bit
    for i in range(16):
        input_bit = int(input_bits[i])
        key_bit = int(key_bits[i])
        result_bit = input_bit ^ key_bit  # XOR operation
        result += str(result_bit)
    return bits_to_hex(result)

### Mixing Columns func

In [40]:
def inverse_mix_columns(input_hex, matrix): # here the matrix will be inverse constart which was 9229
    # Convert the hexadecimal characters to integers
    c0 = int(input_hex[0], 16)
    c1 = int(input_hex[1], 16)
    c2 = int(input_hex[2], 16)
    c3 = int(input_hex[3], 16)
    # Perform the MixColumns operation
    d0 = format(gf_multiply(matrix[0][0], c0) ^ gf_multiply(matrix[0][1], c1), '04x')
    d1 = format(gf_multiply(matrix[1][0], c0) ^ gf_multiply(matrix[1][1], c1), '04x')
    d2 = format(gf_multiply(matrix[0][0], c2) ^ gf_multiply(matrix[0][1], c3), '04x')
    d3 = format(gf_multiply(matrix[1][0], c2) ^ gf_multiply(matrix[1][1], c3), '04x')
    # Combine the results
    output_hex = d0[3]+d1[3]+d2[3]+d3[3]
    return output_hex

## Shift row function

In [41]:
def inverse_shift_row(input_hex):
    # Swap the first nibble  with the third nibble
    return input_hex[2]+input_hex[1]+input_hex[0]+input_hex[3]

### Generating round keys

In [42]:
def GenerateRoundKeys(master_key):
    # Ensure the input master_key has exactly 4 hexadecimal characters
    if len(master_key) != 4:
        raise ValueError("Master key must have 4 hexadecimal characters")
    # Extract the individual nibbles (w0, w1, w2, w3)
    w0, w1, w2, w3 = [int(char, 16) for char in master_key]
    # Calculate w4 to w7
    w4 = w0 ^ int(substitution_table[format(w3, 'x')], 16) ^ int(rcon1, 2)
    w5 = w1 ^ w4
    w6 = w2 ^ w5
    w7 = w3 ^ w6
    # Calculate w8 to w11
    w8 = w4 ^ int(substitution_table[format(w7, 'x')], 16) ^ int(rcon2, 2)
    w9 = w5 ^ w8
    w10 = w6 ^ w9
    w11 = w7 ^ w10
    # Format the results as 4-character hexadecimal strings
    round_key1 = format(w4, '04x')[3] + format(w5, '04x')[3] + format(w6, '04x')[3] + format(w7, '04x')[3]
    round_key2 = format(w8, '04x')[3] + format(w9, '04x')[3] + format(w10, '04x')[3] + format(w11, '04x')[3]
    return round_key1, round_key2

### Main decryption function 

The followup for decryption is the inverse of the encryption function

In [43]:
def Decryption(cipher, key):
    
    # generating the round keys
    roundKeys = GenerateRoundKeys(key)
    print(roundKeys)
    # step2 
    # applying shiftrow function to shift the text back to orignal characters
    text = inverse_shift_row(cipher)
    print(text)
    #step 3
    #applying add round_key function to add the round key to the text 
    text = AddRoundKeys(text, roundKeys[1])
    print(text)
    # step 4 
    # apply substituion algorithm to the nibbles
    text = inverse_subnibbles(text, reverse_substitution_table)
    print(text)
    #step 5 
    # shifting the row second time
    text = inverse_shift_row(text)
    print(text)
    # step6
    # inverse column mixing for decryption
    text = inverse_mix_columns(text, inverse_constant_matrix)
    print(text)
    #step7
    #applying add round_key function to add the round key to the text
    text = AddRoundKeys(text, roundKeys[0])
    print(text)
    #finalling the decryption
    text = inverse_subnibbles(text, reverse_substitution_table)
    return text

### Main

In [44]:
cipher_text = input("Enter the ciphertext block : ")
master_key = input("Enter key : ")
print("Entered ciphertext block : ", cipher_text)
print("Entered Key : ", master_key)

print("----------------------------------------------------------------\n\n")
print("Decrypted block : ", Decryption(cipher_text, master_key))

Entered ciphertext block :  8271
Entered Key :  ea77
----------------------------------------------------------------


('5f8f', '780f')
7281
0a8e
10f3
f013
edfa
b275
Decrypted block :  ceb7


# Question number 3

In [45]:
def Decrypt_File(filename, key):
    decrypted_hex = ""
    # Open the file for reading
    with open('secret.txt', 'r') as file:
        # Loop through each line in the file
        for line in file:
            # Split the line into ciphers using spaces as the delimiter
            ciphers = line.split()
            # Loop through each cipher in the line
            for cipher in ciphers:
                # Decrypt the cipher
                decrypted_hex += Decryption(cipher, key)
    # if last two characters are 00, remove them            
    if decrypted_hex[-2:] == "00":
      decrypted_hex = decrypted_hex[:-2]
    # Convert the hexadecimal string to bytes
    byte_string = bytes.fromhex(decrypted_hex)
    # Decode the bytes into a UTF-8 string
    english_string = byte_string.decode('utf-8')
    return english_string

In [46]:
filename = "secret.txt"
master_key = input("Enter key : ")
print()
plain_text = Decrypt_File(filename, master_key)
# write the plain text on new file and name it with decrypted.txt
with open('decrypted.txt', 'w') as file:
    file.write(plain_text)
print("Decrypted Result: ", plain_text)


('4095', 'dd41')
9975
4434
dd5d
5ddd
2566
65f3
('4095', 'dd41')
7cae
a1ef
0836
3806
b2c3
f256
('4095', 'dd41')
9d45
4004
d11d
11dd
bb66
fbf3
('4095', 'dd41')
9f55
4214
de8d
8edd
b466
f4f3
('4095', 'dd41')
9ca3
41e2
d83e
38de
b20e
f29b
('4095', 'dd41')
1aa5
c7e4
ab3d
3bad
dac8
9a5d
('4095', 'dd41')
840f
594e
72d3
d273
b8c6
f853
('4095', 'dd41')
6aab
b7ea
cb30
3bc0
da6b
9afe
('4095', 'dd41')
013a
dc7b
9abc
ba9c
b067
f0f2
('4095', 'dd41')
7b8e
a6cf
04a6
a406
d5c3
9556
('4095', 'dd41')
caa1
17e0
8b31
3b81
da6a
9aff
('4095', 'dd41')
2886
f5c7
67ab
a76b
bd60
fdf5
('4095', 'dd41')
7ace
a78f
0bf6
fb06
b1c3
f156
('4095', 'dd41')
7aaf
a7ee
0b33
3b03
da68
9afd
('4095', 'dd41')
0ca9
d1e8
983f
389f
b20f
f29a
('4095', 'dd41')
9ac5
4784
dbfd
fbdd
b166
f1f3
('4095', 'dd41')
99a5
44e4
dd3d
3ddd
1966
59f3
('4095', 'dd41')
0379
de38
935f
539f
d20f
929a
('4095', 'dd41')
e0f9
3db8
59cf
c95f
7664
36f1
('4095', 'dd41')
2883
f5c2
67ae
a76e
bdcb
fd5e
('4095', 'dd41')
7aaf
a7ee
0b33
3b03
da68
9afd
('4095', 'dd